In [798]:
import numpy as np
import math

t = 0  # number of this round
T = 8000  # numbers of rounds
l = 30  # ||a|| <= L
d = 3 #dimension of arm
m = 1 #||theta|| < m
theta_stimato = [0.0, 0.0, 0.0] #np.array([])  theta
A = [] #collector of arm
num_arms = 0 #number of arms
num_theta = 0 #num theta
Th = [] #collector of theta
W = [] #distribution of probability to play arms
N = [] #number of times an arm is played
R = [] #collection of reward
H = [] #collection of hedge losses
L = [] #collection of losses
Mixability_Gap = [] #collection of all mixability gap at each t
AI = [] #number of the arm played at time I
nu = 0.01 #regularization paramether
eta = 0.8 #learning paramether
theta_vero = [0]*6
curr_answ = 0

sample_correct = [0]*800
error_reward = [0]*800
error_best_choosen = [0] * 800

Y = []
num_arms_y = 0

M = [] #world
num_M = 0

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1
        
class arm_y():
    arm_vector = np.array([])
    id = 0

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, arm_Y
        Y.append(self)
        self.id = num_arms_y
        num_arms_y += 1
        
def create_theta():
    for i in range (0, num_arms_y):
        new_theta = theta(np.matmul(M[0].matrix, Y[i].arm_vector))

def normalize(v):
    sum = np.sum(v)
    if sum > 0:
        return v / sum
    else:
        return np.array([0.33]*len(v))
    
class theta():
    vector = np.array([])
    best_arm = -1
    id

    def __init__(self, param):
        self.vector = param
        global Th, num_theta
        self.id = num_theta
        Th.append(self)
        num_theta +=1
        #i = 0
        #best = -5
        #while i < num_arms:
            #tmp = np.dot( A[i].arm_vector, np.transpose(self.vector))
            #if best < tmp:
                #best = tmp
                #self.best_arm = i

class arm():
    arm_vector = np.array([])
    id = 0
    best_theta = -1
    regret_y = []

    def __init__(self, vector):
        self.regret_y = [0] *num_arms_y
        self.arm_vector = vector
        global num_arms, A
        A.append(self)
        W.append(0)
        N.append(0)
        L.append(1)
        self.id = num_arms
        num_arms +=1
        max = -5
        for i in range (0, num_theta):
            tmp = np.matmul(self.arm_vector, Th[i].vector)
            if tmp > max:
                self.best_theta = i
                max = tmp
                            
    def new_theta_vero(self):
        min = 1000000000000000
        curr = -1
        for i in range (0, num_arms_y):
            if self.regret_y[i] < min:
                curr = i
                min = self.regret_y[i]
        global theta_vero
        theta_vero[self.id] = curr
                    
    def pull_arm(self):
        r = np.dot(self.arm_vector, np.transpose(Th[theta_vero[self.id]].vector)) + np.random.normal(0, 1)
        R.append(r)
        #print("self.id", self.id)
        #print("theta_vero", theta_vero)
        #print("regret_y", self.regret_y)
        self.regret_y[theta_vero[self.id]] += (4*(l**2)*(m**2) - r)/1000
        self.new_theta_vero()
        N[self.id] += 1
        AI.append(self.id)
        return r
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

def design_matrix(w):
    V = []
    i = 0
    while i < num_arms:
        b = [A[i].arm_vector]
        a = np.swapaxes(b, 0, 1)
        ris = np.matmul(a, b) * w[i]
        if (i == 0):
            V = ris
        else:
            V += ris
        i+=1
    return V

def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

def arm_to_play():
    diff = np.array(N) - np.array(W)
    i = 0
    arm = 0
    min = +5000
    while i < num_arms:
        if diff[i] < min:
            min = diff[i]
            arm = i
        i += 1
    return arm

def stima_theta():
    V = design_matrix(N)
    I = np.identity(d)
    p = np.linalg.inv(V + nu*I)
    sum = 0
    for i in range (0, t):
        sum += R[i] * np.array(A[AI[i]].arm_vector)
    #print("sum", sum)
    #print("p", p)
    res = np.matmul(p, sum)
    #print("res", res)
    global theta_stimato
    theta_stimato = res
    return res

def best_answer():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, theta_stimato)
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best

def best_arm():
    max = -100
    best = -1
    for i in range (0, num_arms):
        for j in range (0, num_theta):
            min = 1000000000
            curr = np.matmul(A[i].arm_vector, Th[j].vector)
            if curr < min:
                min = curr
        if min > max:
            best = i
            max = min
    global curr_answ
    curr_answ = best
    return best

def best_value():
    max = -100
    for i in range (0, num_arms):
        for j in range (0, num_theta):
            min = 1000000000
            curr = np.matmul(A[i].arm_vector, Th[j].vector)
            if curr < min:
                min = curr
        if min > max:
            max = min
    return max


def beta(time, delta):
    n = np.abs(np.random.normal(0, 1))
    res = math.sqrt(math.log10(1/delta) + d/2*math.log10(1 + (time*(l**2))/(n*d))) + math.sqrt(n/2)*m
    return res

def best_response():
    min = 1000000000000
    theta_adv = 0
    for i in range (0, num_theta):
        if not i == A[curr_answ].best_theta:
            curr = (norm_2_mat(np.array(theta_stimato) - np.array(Th[i].vector), design_matrix(W)))**2
            if curr < min:
                min = curr
                theta_adv = i
    return theta_adv

def calcola_U (id_arm):
    xi = [0]*d
    q2 = 4*(l**2)*(m**2)
    q1 = 0
    #print("theta_stimato", theta_stimato)
    #print("best_response()", best_response())
    #print("id_arm", id_arm)
    p1 = np.matmul((np.array(theta_stimato) - np.array(Th[best_response()].vector)), np.array(A[id_arm].arm_vector))
    p2 = math.sqrt(2*beta(T, 1/T**3))*norm_2_mat(np.array(A[id_arm].arm_vector), design_matrix(normalize(N)) + np.random.normal(0, 1)*np.identity(d))
    r1 = (p1 + p2)**2
    r2 = (p1 - p2)**2
    if r1 > r2:
        q1 = r1
    else:
        q1 = r2
    if q1 > q2:
        return q2
    else:
        return q1

def calcola_loss():
    loss = 0
    par = 0
    arm_play = arm_to_play()
    best_res = best_response ()
    for i in range (0, num_arms):
        #print("IMPORTANTE: W", W)
        #print("W[i]", W[i])
        #print("calcola_U(i)", calcola_U(i))
        par = W[i]*calcola_U(i)/2
        if math.isnan(par):
            loss += 0.1
        else:
            loss += par
        #print("loss parziale i", loss, i)
    #print("loss", loss)
    #print("lm etc", 4*(l**2)*(m**2))
    L[arm_play] += (4*(l**2)*(m**2) - loss)/1000
    return (4*(l**2)*(m**2) - loss)/1000

In [799]:
len([0,1])

2

In [800]:
ze = arm_y( [0.8705524572652754, 0.0619707940889308, 0.6618039107464511] )
ke = arm_y( [0.29313747095524345, 0.7147227382224571, 0.43880766191059795] )
le = arm_y( [0.3976778652509695, 0.20770310609484777, 0.791678366091227] )

In [801]:
wor = world( [[0.5899011009128065, 0.5657184902992989, 0.6054951466017203], [0.82708485598313, 0.1528251973706528, 0.4138971668624878], [0.4974849256226015, 0.7843065267705409, 0.12423891479137728]] )

In [802]:
create_theta()

In [803]:
ae = arm( [0.9871376736199118, 0.688121853533161, 0.5235809151333978] )
be = arm( [0.03481376451873974, 0.34685413538499243, 0.05591468294799695] )
ce = arm( [0.8484788173613923, 0.9117151239295301, 0.43862231513255734] )
de = arm( [0.6311689207509968, 0.08306431034585382, 0.49042361557796177] )

In [804]:
num_theta

3

In [805]:
Th[2].vector

array([0.83144951, 0.68832904, 0.45909891])

In [806]:
w = [0.25, 0.25, 0.25, 0.25]
V = design_matrix(w)

In [807]:
for i in range (0, num_arms):
    A[i].pull_arm()

In [808]:
for i in range (0, num_arms):
    #print(A[i].best_theta)

0
0
0
0


In [809]:
for i in range(0, num_arms):
    for j in range (0, num_theta):
        #print(np.matmul(A[i].arm_vector, Th[j].vector))
#print(best_arm())
#print(best_value())

1.9228288935043807
1.5974799618787874
1.534784812354001
0.4126172746467014
0.2568690843497489
0.29336603143560747
1.9676442382309933
1.5351935764319555
1.534398315792654
0.9590829887935023
0.9495101599700522
0.8071136101287715
0
1.534784812354001


In [810]:
arm_to_play()

0

In [811]:
for i in range (0, num_arms):
    #print(np.matmul(A[i].arm_vector, Th[theta_vero[i]].vector))

1.5974799618787874
0.2568690843497489
1.5351935764319555
0.9495101599700522


In [812]:
def h():
    return np.dot(W, L)

def mix_loss():
    sum = 0
    for i in range (0, num_arms):
        sum += W[i] * math.exp(-eta * L[i])
    return -1/eta * np.log(sum)

def mixability_gap():
    m_g = h() - mix_loss()
    Mixability_Gap.append(m_g)

def new_eta():
    if t>0:
        sum = 0
        for i in range (0, t):
            sum += Mixability_Gap[i]
        eta = math.log(d)/sum
        
def generate_w():
    sum=0
    for i in range (0, num_arms):
        W[i] = math.exp(-eta * L[i])/num_arms
        sum += math.exp(-eta * L[i])/num_arms
    for i in range (0, num_arms):
        W[i] = W[i]/sum
    return W

In [813]:
W

[0, 0, 0, 0]

In [814]:
t += 1
stima_theta()
best_answer()
curr = arm_to_play()
mixability_gap()
generate_w()
err = calcola_loss()
#print(err)
A[curr].pull_arm()

sum [1.6183     1.12809756 0.85835139]
p [[ 17.09806091  -5.04992666 -22.74967552]
 [ -5.04992666   3.46174575   4.47490195]
 [-22.74967552   4.47490195  34.21426989]]
res [ 2.44576644 -0.42607108 -2.39980781]
theta_stimato [ 2.44576644 -0.42607108 -2.39980781]
best_response() 2
id_arm 0
theta_stimato [ 2.44576644 -0.42607108 -2.39980781]
best_response() 2
id_arm 1
theta_stimato [ 2.44576644 -0.42607108 -2.39980781]
best_response() 2
id_arm 2
theta_stimato [ 2.44576644 -0.42607108 -2.39980781]
best_response() 2
id_arm 3
3.596537820009926


D:\Programmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


0.8065565953745695

In [815]:
t += 1
stima_theta()
best_answer()
curr = arm_to_play()
mixability_gap()
new_eta()
generate_w()
err = calcola_loss()
#print(err)
A[curr].pull_arm()

sum [1.60771793 1.02266699 0.84135544]
p [[ 15.64088418  -4.79607541 -21.31988073]
 [ -4.79607541   3.41752294   4.22582084]
 [-21.31988073   4.22582084  32.81134251]]
res [ 2.30374436 -0.6603312  -2.34874562]
theta_stimato [ 2.30374436 -0.6603312  -2.34874562]
best_response() 2
id_arm 0
theta_stimato [ 2.30374436 -0.6603312  -2.34874562]
best_response() 2
id_arm 1
theta_stimato [ 2.30374436 -0.6603312  -2.34874562]
best_response() 2
id_arm 2
theta_stimato [ 2.30374436 -0.6603312  -2.34874562]
best_response() 2
id_arm 3
3.587118685285068


1.1087287948906928

In [816]:
for i in range (800):
    t += 1
    stima_theta()
    best_answer()
    curr = arm_to_play()
    mixability_gap()
    new_eta()
    generate_w()
    err = calcola_loss()
    #print("err", err)
    A[curr].pull_arm()
    if best_answer() == best_arm():
        sample_correct[i] += 1
    error_reward[i] += best_value() - np.dot(A[best_arm()].arm_vector, np.transpose(stima_theta()))
    error_best_choosen[i] += best_value() - np.dot(A[best_answer()].arm_vector, np.transpose(stima_theta()))

sum [2.81068928 2.31529463 1.46323317]
p [[ 12.0747643   -2.86003868 -17.37257218]
 [ -2.86003868   2.3664539    2.08283783]
 [-17.37257218   2.08283783  28.44209937]]
res [ 1.8964546   0.48803532 -2.38909608]
theta_stimato [ 1.8964546   0.48803532 -2.38909608]
best_response() 2
id_arm 0
theta_stimato [ 1.8964546   0.48803532 -2.38909608]
best_response() 2
id_arm 1
theta_stimato [ 1.8964546   0.48803532 -2.38909608]
best_response() 2
id_arm 2
theta_stimato [ 1.8964546   0.48803532 -2.38909608]
best_response() 2
id_arm 3
err 3.591769244516236
sum [2.81068928 2.31529463 1.46323317]
p [[ 12.07454842  -2.86792358 -17.36809442]
 [ -2.86792358   2.07845617   2.24638915]
 [-17.36809442   2.24638915  28.34922005]]
res [ 1.88414399  0.03838746 -2.1337451 ]
sum [2.81068928 2.31529463 1.46323317]
p [[ 12.07454842  -2.86792358 -17.36809442]
 [ -2.86792358   2.07845617   2.24638915]
 [-17.36809442   2.24638915  28.34922005]]
res [ 1.88414399  0.03838746 -2.1337451 ]
sum [4.10792616 2.4860161  2.471

D:\Programmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:153: RuntimeWarning: invalid value encountered in sqrt


1
id_arm 3
err 3.5984087944586975
sum [17.5738534  12.7511325  10.55528806]
p [[ 5.03311499 -1.36757716 -6.85877901]
 [-1.36757716  0.71382399  1.47305151]
 [-6.85877901  1.47305151 10.04263851]]
res [-1.38332079  0.61694676  4.25084035]
sum [17.5738534  12.7511325  10.55528806]
p [[ 5.03311499 -1.36757716 -6.85877901]
 [-1.36757716  0.71382399  1.47305151]
 [-6.85877901  1.47305151 10.04263851]]
res [-1.38332079  0.61694676  4.25084035]
sum [18.94288384 13.70546723 11.2814261 ]
p [[ 5.03311499 -1.36757716 -6.85877901]
 [-1.36757716  0.71382399  1.47305151]
 [-6.85877901  1.47305151 10.04263851]]
res [-0.77837985  0.49555775  3.55908909]
theta_stimato [-0.77837985  0.49555775  3.55908909]
best_response() 1
id_arm 0
theta_stimato [-0.77837985  0.49555775  3.55908909]
best_response() 1
id_arm 1
theta_stimato [-0.77837985  0.49555775  3.55908909]
best_response() 1
id_arm 2
theta_stimato [-0.77837985  0.49555775  3.55908909]
best_response() 1
id_arm 3
err 3.5942178621292613
sum [18.9428838

best_response() 1
id_arm 0
theta_stimato [-1.62966421  1.21549646  4.06341403]
best_response() 1
id_arm 1
theta_stimato [-1.62966421  1.21549646  4.06341403]
best_response() 1
id_arm 2
theta_stimato [-1.62966421  1.21549646  4.06341403]
best_response() 1
id_arm 3
err 3.590219946564144
sum [27.13608925 20.9090112  15.97418873]
p [[ 3.72843651 -1.04416033 -5.06788519]
 [-1.04416033  0.5252252   1.15306139]
 [-5.06788519  1.15306139  7.35624053]]
res [-1.61252872  1.06673186  4.09676327]
sum [27.13608925 20.9090112  15.97418873]
p [[ 3.72843651 -1.04416033 -5.06788519]
 [-1.04416033  0.5252252   1.15306139]
 [-5.06788519  1.15306139  7.35624053]]
res [-1.61252872  1.06673186  4.09676327]
sum [27.19375869 21.48357948 16.06681212]
p [[ 3.72843651 -1.04416033 -5.06788519]
 [-1.04416033  0.5252252   1.15306139]
 [-5.06788519  1.15306139  7.35624053]]
res [-2.46685795  1.41509391  5.14837354]
theta_stimato [-2.46685795  1.41509391  5.14837354]
best_response() 1
id_arm 0
theta_stimato [-2.46685

sum [32.51985393 25.35023937 19.39833877]
p [[ 2.89213588 -0.78341729 -3.96282214]
 [-0.78341729  0.39932526  0.85867488]
 [-3.96282214  0.85867488  5.80888248]]
res [-2.68014567  1.30324117  5.57988684]
sum [35.00645866 27.08362175 20.71724171]
p [[ 2.89213588 -0.78341729 -3.96282214]
 [-0.78341729  0.39932526  0.85867488]
 [-3.96282214  0.85867488  5.80888248]]
res [-2.0730864   1.17988421  4.87567866]
theta_stimato [-2.0730864   1.17988421  4.87567866]
best_response() 1
id_arm 0
theta_stimato [-2.0730864   1.17988421  4.87567866]
best_response() 1
id_arm 1
theta_stimato [-2.0730864   1.17988421  4.87567866]
best_response() 1
id_arm 2
theta_stimato [-2.0730864   1.17988421  4.87567866]
best_response() 1
id_arm 3
err 3.5987641407440125
sum [35.00645866 27.08362175 20.71724171]
p [[ 2.83723351 -0.7722609  -3.89913363]
 [-0.7722609   0.39705823  0.84573311]
 [-3.89913363  0.84573311  5.73500177]]
res [-2.37341837  1.24091291  5.22407336]
sum [35.00645866 27.08362175 20.71724171]
p [[ 2.

...

err 3.596646158900788
sum [683.45687046 530.92232537 388.7114255 ]
p [[ 0.18015665 -0.04868206 -0.24824779]
 [-0.04868206  0.02410509  0.05445345]
 [-0.24824779  0.05445345  0.36411373]]
res [0.78615728 0.69252538 0.77905931]
sum [683.45687046 530.92232537 388.7114255 ]
p [[ 0.18015665 -0.04868206 -0.24824779]
 [-0.04868206  0.02410509  0.05445345]
 [-0.24824779  0.05445345  0.36411373]]
res [0.78615728 0.69252538 0.77905931]
sum [685.36829315 532.97620455 389.69953814]
p [[ 0.18015665 -0.04868206 -0.24824779]
 [-0.04868206  0.02410509  0.05445345]
 [-0.24824779  0.05445345  0.36411373]]
res [0.78522895 0.70278849 0.77617903]
theta_stimato [0.78522895 0.70278849 0.77617903]
best_response() 1
id_arm 0
theta_stimato [0.78522895 0.70278849 0.77617903]
best_response() 1
id_arm 1
theta_stimato [0.78522895 0.70278849 0.77617903]
best_response() 1
id_arm 2
theta_stimato [0.78522895 0.70278849 0.77617903]
best_response() 1
id_arm 3
err 3.5926682180059086
sum [685.36829315 532.97620455 389.6995

best_response() 1
id_arm 0
theta_stimato [0.85959544 0.67201145 0.68188144]
best_response() 1
id_arm 1
theta_stimato [0.85959544 0.67201145 0.68188144]
best_response() 1
id_arm 2
theta_stimato [0.85959544 0.67201145 0.68188144]
best_response() 1
id_arm 3
err 3.5983012891104527
sum [693.26064684 538.61428642 394.01701116]
p [[ 0.17748632 -0.04789691 -0.24468561]
 [-0.04789691  0.02373143  0.05355055]
 [-0.24468561  0.05355055  0.35911518]]
res [0.83602381 0.67686977 0.70967545]
sum [693.26064684 538.61428642 394.01701116]
p [[ 0.17748632 -0.04789691 -0.24468561]
 [-0.04789691  0.02373143  0.05355055]
 [-0.24468561  0.05355055  0.35911518]]
res [0.83602381 0.67686977 0.70967545]
sum [695.23246164 538.873785   395.54912802]
p [[ 0.17748632 -0.04789691 -0.24468561]
 [-0.04789691  0.02373143  0.05355055]
 [-0.24468561  0.05355055  0.35911518]]
res [0.79867783 0.6706299  0.79130345]
theta_stimato [0.79867783 0.6706299  0.79130345]
best_response() 1
id_arm 0
theta_stimato [0.79867783 0.670629

theta_stimato [0.75571311 0.68003575 0.85550521]
best_response() 1
id_arm 0
theta_stimato [0.75571311 0.68003575 0.85550521]
best_response() 1
id_arm 1
theta_stimato [0.75571311 0.68003575 0.85550521]
best_response() 1
id_arm 2
theta_stimato [0.75571311 0.68003575 0.85550521]
best_response() 1
id_arm 3
err 3.5974045743212755
sum [704.76824805 546.26943856 401.21079898]
p [[ 0.17495017 -0.04719973 -0.24115121]
 [-0.04719973  0.02337357  0.05277456]
 [-0.24115121  0.05277456  0.35386109]]
res [0.76308057 0.67711696 0.84629901]
sum [704.76824805 546.26943856 401.21079898]
p [[ 0.17495017 -0.04719973 -0.24115121]
 [-0.04719973  0.02337357  0.05277456]
 [-0.24115121  0.05277456  0.35386109]]
res [0.76308057 0.67711696 0.84629901]
sum [707.26694419 548.01124972 402.53611525]
p [[ 0.17495017 -0.04719973 -0.24115121]
 [-0.04719973  0.02337357  0.05277456]
 [-0.24115121  0.05277456  0.35386109]]
res [0.79841323 0.6698345  0.80463657]
theta_stimato [0.79841323 0.6698345  0.80463657]
best_respons

sum [714.66048172 554.34899048 406.4703855 ]
p [[ 0.17298189 -0.04668071 -0.23847691]
 [-0.04668071  0.02312746  0.05219248]
 [-0.23847691  0.05219248  0.35000174]]
res [0.81211688 0.67452292 0.76816227]
sum [715.05598686 554.40104052 406.77769636]
p [[ 0.17298189 -0.04668071 -0.23847691]
 [-0.04668071  0.02312746  0.05219248]
 [-0.23847691  0.05219248  0.35000174]]
res [0.80481583 0.67330356 0.78411938]
theta_stimato [0.80481583 0.67330356 0.78411938]
best_response() 1
id_arm 0
theta_stimato [0.80481583 0.67330356 0.78411938]
best_response() 1
id_arm 1
theta_stimato [0.80481583 0.67330356 0.78411938]
best_response() 1
id_arm 2
theta_stimato [0.80481583 0.67330356 0.78411938]
best_response() 1
id_arm 3
err 3.5991282135855607
sum [715.05598686 554.40104052 406.77769636]
p [[ 0.17284681 -0.04670327 -0.23818168]
 [-0.04670327  0.02312369  0.05224178]
 [-0.23818168  0.05224178  0.34935647]]
res [0.81581197 0.67514004 0.76008632]
sum [715.05598686 554.40104052 406.77769636]
p [[ 0.17284681 

id_arm 0
theta_stimato [0.79382208 0.70676246 0.77959459]
best_response() 1
id_arm 1
theta_stimato [0.79382208 0.70676246 0.77959459]
best_response() 1
id_arm 2
theta_stimato [0.79382208 0.70676246 0.77959459]
best_response() 1
id_arm 3
err 3.5926120074335532
sum [731.03606378 568.44137537 415.66182098]
p [[ 0.17025611 -0.0460004  -0.23461466]
 [-0.0460004   0.02275749  0.05146322]
 [-0.23461466  0.05146322  0.34412482]]
res [0.79446575 0.6996402  0.78159463]
sum [731.03606378 568.44137537 415.66182098]
p [[ 0.17025611 -0.0460004  -0.23461466]
 [-0.0460004   0.02275749  0.05146322]
 [-0.23461466  0.05146322  0.34412482]]
res [0.79446575 0.6996402  0.78159463]
sum [731.06911932 568.77071213 415.71491177]
p [[ 0.17025611 -0.0460004  -0.23461466]
 [-0.0460004   0.02275749  0.05146322]
 [-0.23461466  0.05146322  0.34412482]]
res [0.77248816 0.70834673 0.80905791]
theta_stimato [0.77248816 0.70834673 0.80905791]
best_response() 1
id_arm 0
theta_stimato [0.77248816 0.70834673 0.80905791]
bes

In [817]:
#print(theta_stimato)
#print(Th[theta_vero[best_answer()]].vector)
#print(best_answer())
#print(best_arm())
#print(W)

[0.76734997 0.68864169 0.83512594]
[0.94931693 1.00341022 0.56391262]
0
0
[0.01867023724499146, 0.26435205148125446, 0.4087286108487782, 0.30824910042497594]


In [818]:
#print(theta_vero)

[0, 1, 2, 0, 0, 0]


In [819]:
R

[1.6393863227110328,
 -0.3039622614086742,
 1.4177977439235163,
 2.055292696617034,
 0.8065565953745695,
 1.1087287948906928,
 1.3206134833877083,
 0.9788961717437598,
 3.3076675272983644,
 -0.022014428433700328,
 1.124785769637388,
 1.4155790332216132,
 0.8400736172234187,
 1.1426713892894986,
 1.7610601439064346,
 1.8721736246292597,
 0.7652106198366248,
 0.4449889525875952,
 2.408230161002401,
 1.3868688067042199,
 1.304162557697834,
 0.6991949134834405,
 1.517103685665659,
 2.973507631915052,
 1.5270312757943247,
 1.4222285001524348,
 1.3316709451580087,
 0.7729284306401245,
 -1.0064893928330645,
 1.7019883870433254,
 1.6565127093350982,
 0.3761824266488729,
 1.8634215467330284,
 1.6841979981984976,
 1.3478806022520353,
 1.106539423774647,
 1.166470849944385,
 0.547913803324089,
 0.2655946583239268,
 2.519004991797327,
 2.1940172455509273,
 1.4052225944470829,
 1.4936386612620274,
 0.762335733190676,
 1.8803651052041133,
 2.971480929988515,
 0.46982777564724576,
 1.5867638259518526

In [820]:
N

[201, 201, 202, 202]

In [821]:
#print(sample_correct)

[0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [822]:
#print(error_reward)

[0.7656482605275898, 0.6056941523443753, 0.4421278949335927, 0.523546818648184, 0.46189257380563675, 0.6174488739125473, 0.5264716462016141, 0.513080390157709, 0.47372580597945113, 0.4222737409580197, 0.44074903206546767, 0.4846339079539148, 0.41243314868580283, 0.32448696377896646, 0.33809187051977996, 0.43694441268656314, 0.2501194477455042, 0.24005784252314255, 0.2393421868233978, 0.2722862254610068, 0.3147977951180967, 0.16603430516230433, 0.16268567091175612, 0.15877026217316192, 0.18807178773393085, 0.2308426211311585, 0.26213424388330786, 0.2475440965289506, 0.2802302198367794, 0.33862372510766203, 0.3292823871387367, 0.3173715550812364, 0.33535304637465213, 0.34092597310202044, 0.34070530980921543, 0.3726854591579021, 0.3621465838629081, 0.28855109608945817, 0.2787779771184149, 0.27863012058592806, 0.2975816180946398, 0.32142110881812247, 0.3159276038856276, 0.2676093998862399, 0.261955227865589, 0.2473836186753866, 0.24066373327775414, 0.22008182109973173, 0.22449664063384134,

In [823]:
#print(error_best_choosen)

[0.7656482605275898, 0.5563780101253377, 0.4421278949335927, 0.523546818648184, 0.46189257380563675, 0.5945669958434407, 0.048834030502862724, 0.047468981497109874, 0.054181404340937656, 0.04185512846864148, 0.16333734245328668, 0.1645818396775811, 0.17809013550058084, 0.1583141310907732, 0.2537340035856068, 0.23735627211471355, 0.2501194477455042, 0.24005784252314255, 0.2393421868233978, 0.2576147226501324, 0.25493755575139865, 0.16603430516230433, 0.16268567091175612, 0.15877026217316192, 0.15328189807824755, 0.18103167739630432, 0.15633962777526778, 0.13349391404017563, 0.10021323223369283, 0.13797447859917034, 0.14551249560642554, 0.12684903841732664, 0.10854993194543572, 0.11214409867395592, 0.11232550911676453, 0.1624643973736397, 0.17318407113856216, 0.12582580270269172, 0.13398781044149088, 0.13375589234455565, 0.11448830343209448, 0.12979995805796585, 0.13445007283096522, 0.0586322147734224, 0.06437820068498556, 0.055034232404510774, 0.06078920155100409, 0.028482888072554724, 

In [824]:
import numpy as np        
f=open('087029GAM.txt','a')
np.savetxt(f, error_reward, fmt='%1.3f', newline=", ")
f.write("\n")
np.savetxt(f, error_best_choosen, fmt='%1.3f', newline=", ")
f.write("\n")
np.savetxt(f, sample_correct, fmt='%1.3f', newline=", ")
f.write("\n")
f.close()